In [2]:
import os
import pandas as pd
import collections.abc
import numpy as np
from functools import reduce  # forward compatibility for Python 3
import operator
from scipy.stats import norm
import os
#os.chdir('/users/hollowayp/paltas/')

In [6]:
#[0] zl, lens redshift 
#[1] zs, source redshift 
#[2] b, Einstein radius (arcseconds) 
#[3] sig_v, lens velocity dispersion (km/s) 
#[4] ql, lens flattening (1=circular, q=1-e) 
#[5] rl, lens half light radius (arcseconds) NB/ sersic index is 4
#[6] lens g band magnitude
#[7] lens r band magnitude
#[8] lens i band magnitude
#[9] xs, source x coordinate relative, 0 is lens centre (arcseconds) 
#[10] ys, source y coordinate relative, 0 is lens centre (arcseconds) 
#[11] qs, source flattening (1=circular, q=1-e) 
#[12] ps, source position angle (degrees)
#[13] rs, source half light radius (arcseconds) NB/ sersic index is 1
#[14] source g band magnitude
#[15] source r band magnitude
#[16] source i band magnitude
#[17] mu_s, source magnification
#[18] g band coadd seeing
#[19] g band coadd signal-to-noise of source assuming poisson limited lens subtraction
#[20] r band coadd seeing
#[21] r band coadd signal-to-noise of source assuming poisson limited lens subtraction
#[22] i band coadd seeing
#[23] i band coadd signal-to-noise of source assuming poisson limited lens subtraction
#[24] signal-to-noise in the g - i difference image
from Load_LensPop_LSST_db import db_LensPop_LSST as db_LSST


In [19]:
class generate_config_file:
     '''
     Generates a new configuration file, based on a template config (located at old_config_file_path)
     which is updated using the values in the new_config_dict_values dictionary.

     #### An important note on formatting:
     The template config file must be formatted in a specific way for this config-generator to work: 
     1) All '}' must be on their own line (comma's don't matter), i.e. 
     'main_deflector':{
     'class': PEMDShear
     }
     is ok, but:
     'main_deflector':{
     'class': PEMDShear}
     is not.
     #### End of note.

     '''
     def __init__(self,old_config_file_path,new_config_file_path):
        #
#        self.config_dict = config_dict_0
        self.old_config_file_path = old_config_file_path
        self.new_config_file_path = new_config_file_path
        random_db_indx = np.random.randint(len(db_LSST))
        self.zL = db_LSST['zL'][random_db_indx]
        self.zS = db_LSST['zS'][random_db_indx]
        self.mag_app = db_LSST['i_source'][random_db_indx] #i-band apparent source magnitude
        self.tE = db_LSST['tE'][random_db_indx]
        self.q_source = db_LSST['q_source_flat'][random_db_indx]
        self.PA_source = db_LSST['PA_source'][random_db_indx]
        self.Re_source = db_LSST['Re_source'][random_db_indx]
        self.x_source = db_LSST['xs'][random_db_indx]
        self.y_source = db_LSST['ys'][random_db_indx]
        self.e1_source = db_LSST['e1_source'][random_db_indx]
        self.e2_source = db_LSST['e2_source'][random_db_indx]
        self.new_config_dict_values = {
             'main_deflector':{
                  'parameters':{
                       'z_lens':self.zL,
                       'theta_E':self.tE,
                       'center_x':0.0,
                       'center_y':0.0
                  }
             },
             'source':{
                  'parameters':{
                       'z_source':self.zS,
                       'mag_app':self.mag_app,
                       'R_sersic':self.Re_source,
                       'n_sersic':1.0, #Fixed in LensPop
                       'center_x':self.x_source,
                       'center_y':self.y_source,
                       'e1':self.e1_source,
                       'e2':self.e2_source
                  }
             }
        }
     def count_number_of_values_in_dict(self,d):
        return sum([self.count_number_of_values_in_dict(v) if isinstance(v, dict) else 1 for v in d.values()])

     def getFromDict(self,dataDict, mapList):
          return reduce(operator.getitem, mapList, dataDict)

     def write_new_config(self):
          N_new_values = self.count_number_of_values_in_dict(self.new_config_dict_values)
          N_values_changed = 0
          begin_config=False
          key_tracker = []
          with open(self.new_config_file_path,'w') as f_new:
               for line in open(self.old_config_file_path,'r'):
#                    print('LINE',line)
                    if not begin_config:
                         f_new.write(line)
                    if begin_config:
                         line_0 = line.replace('\t','').replace('\n','') #Removing junk
                         if len(line_0)>1:
                              if line_0[0] =='#': 'Continuing'; continue #Ignoring lines which are commented out
                         line_3 = line_0.replace("'",'').split(':')[0]
                         line_4 = line_0.strip().split(':')[0].replace("'",'')
                         if '{' in line_0:
                              key_tracker.append(line_4)
               #			print(key_tracker)
                         if '}' in line_0:
                              assert line_0.replace(',','').strip()=='}'
                              key_tracker = key_tracker[:-1]
               #			print(key_tracker)
                         try:
                              #If the update-dictionary contains these keys (and the corresponding value is a float), the 
                              #value needs to be updated
                              if isinstance(self.getFromDict(self.new_config_dict_values,key_tracker+[line_3]),float):
#                                   print('Writing',"'"+line_3+"':"+
#                                   str(self.getFromDict(self.new_config_dict_values,key_tracker+[line_3]))+",\n")
                                   f_new.write("'"+line_3+"':"+
                                   str(self.getFromDict(self.new_config_dict_values,key_tracker+[line_3]))+",\n")
                                   N_values_changed+=1
                              else:
                                   print('Else',line_0)
                         except Exception as ex:
#                              print('EX',line_0)
                              f_new.write(str(line_0)+'\n')
                              pass
                    if 'config_dict' in line: begin_config=True
          assert N_values_changed==N_new_values #Assert that the same number of value have been updated as in the new_config_dict_values dictionary.  



In [17]:
generate_config_file('./paltas/Configs/Examples/config_LSST.py',
                                          './paltas/Configs/Examples/config_LSST_99.py').write_new_config()

In [77]:
training_directory='pscratch/sd/p/phil1884/Image_Sim_Folders/' #For NERSC
for i in range(10):
    new_config_path_i = f'./paltas/Configs/Examples/config_LSST_{i}.py'
    generate_config_file('./paltas/Configs/Examples/config_LSST.py',
                                          new_config_path_i).write_new_config()
    !python3 ./paltas/generate.py $new_config_path_i /$training_directory/training/LensPop_Config/$i --n 1
    #Moving the image into the parent folder:
    os.rename(f'/pscratch/sd/p/phil1884/Image_Sim_Folders/training/LensPop_Config/{i}/image_0000000.npy',
              f'/pscratch/sd/p/phil1884/Image_Sim_Folders/training/LensPop_Config/image_000000{i}.npy')    

2023-10-16 05:43:44.167790: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-16 05:43:45.031522: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/global/common/software/nersc/pm-2022q3/sw/python/3.9-anaconda-2021.11/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/global/homes/p/phil1884/.local/perlmutter/3.9-anaconda-2021.11/lib/python3.9/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/r

In [7]:
training_directory

NameError: name 'training_directory' is not defined

In [6]:
from plot_image_collage import plot_collage
plot_collage(f'/{training_directory}/training/LensPop_Config/',2,5)


Plotting from h5 files


ValueError: Cannot take a larger sample than population when 'replace=False'

In [ ]:
new_config_dict_values = {
             'main_deflector':{
                  'parameters':{
                       'z_lens':0.3,
                       'theta_E':0.1
                  }}}
from functools import reduce  # forward compatibility for Python 3
import operator

def getFromDict(dataDict, mapList):
    return reduce(operator.getitem, mapList, dataDict)

begin_config=False
key_tracker = []
for line in open('/Users/hollowayp/paltas/paltas/Configs/Examples/config_LSST.py','r'):
	if begin_config:
		line_0 = line.replace('\t','').replace('\n','') #Removing junk
		line_3 = line_0.replace("'",'').split(':')[0]
		line_4 = line_0.strip().split(':')[0].replace("'",'')
		if len(line_0.strip())>1:
			if line_0.strip()[0]=='#': continue #Ignoring lines which are commented out
		if '{' in line_0:
			key_tracker.append(line_4)
			print(key_tracker)
		if '}' in line_0:
			key_tracker = key_tracker[:-1]
#			print(key_tracker)
		try:
			#If the update-dictionary contains these keys (and the corresponding value is a float), the 
			#value needs to be updated
			if isinstance(getFromDict(new_config_dict_values,key_tracker+[line_3]),float):
				print('HEREEEE',key_tracker+[line_3],
		  		getFromDict(new_config_dict_values,key_tracker+[line_3]))
			else:
				print('Else',line_0)
		except Exception as ex:
			print('Ex',line_0)
			pass
	if 'config_dict' in line: begin_config=True

print(key_tracker)

In [10]:
from functools import reduce  # forward compatibility for Python 3
import operator
def generate_config_file_based_on_database(old_config_file_path,new_config_file_path,
                                           list_of_properties,list_of_means,list_of_std,list_of_sigma_to_zero,list_of_truc_bool):
     '''
     Generates a new configuration file, based on a template config (located at old_config_file_path)
     which is updated using the values in the new_config_dict_values dictionary.

     #### An important note on formatting:
     The template config file must be formatted in a specific way for this config-generator to work: 
     1) All '}' must be on their own line (comma's don't matter), i.e. 
     'main_deflector':{
     'class': PEMDShear
     }
     is ok, but:
     'main_deflector':{
     'class': PEMDShear}
     is not.
     2) The 'list_of_properties' should be formatted as key1*key2*key3, where the keys are the keys in the config dictionary.
     #### End of note.
     '''
     new_config_dict_values = {
             'main_deflector':{
                  'parameters':{
                       'z_lens':None,
                       'theta_E':None,
                       'center_x':None,
                       'center_y':None,
			            'gamma': None,
			            'e1': None,
			            'e2': None,
			            'gamma1': None,
			            'gamma2': None,
                  }
             },
            'lens_light':{
                    'parameters':{
                        'z_source' :None,
                        'mag_app':None,
                        'output_ab_zeropoint':None,
                        'R_sersic':None,
                        'n_sersic':None,
                        'e1':None,
                        'e2':None,
                        'center_x':None,
                        'center_y':None
                }
	        },
             'source':{
                  'parameters':{
                       'z_source':None,
                       'mag_app':None,
                       'R_sersic':None,
                       'n_sersic':None,#1.0, #Fixed in LensPop
                       'center_x':None,
                       'center_y':None,
                       'e1':None,
                       'e2':None
                  }
             }
        }
     def generate_new_config_dict():
          for v_i, key_iii in enumerate(list_of_properties):
               key_1,key_2,key_3 = key_iii.split('*')
               if list_of_std[v_i]==0:
                    new_config_dict_values[key_1][key_2][key_3] = list_of_means[v_i]
               elif list_of_truc_bool[v_i]==False:
                    new_config_dict_values[key_1][key_2][key_3] = f'norm(loc={list_of_means[v_i]},scale={list_of_std[v_i]}).rvs'
               else:
                    if list_of_sigma_to_zero[v_i]==-np.inf:
                         list_of_sigma_to_zero[v_i]='-np.inf'
                    new_config_dict_values[key_1][key_2][key_3] = f'truncnorm({list_of_sigma_to_zero[v_i]},np.inf,'+\
                                                                  f'loc={list_of_means[v_i]},scale={list_of_std[v_i]}).rvs'
          return new_config_dict_values

     new_config_dict_values = generate_new_config_dict()
     print(generate_new_config_dict())
#
     def count_number_of_values_in_dict(d):
        return sum([count_number_of_values_in_dict(v) if isinstance(v, dict) else 1 for v in d.values()])

     def getFromDict(dataDict, mapList):
          return reduce(operator.getitem, mapList, dataDict)

     def write_new_config():
          N_new_values = len(list_of_properties)
          N_values_changed = 0
          begin_config=False
          key_tracker = []
          with open(new_config_file_path,'w') as f_new:
               for line in open(old_config_file_path,'r'):
#                    print('LINE',line)
                    if not begin_config:
                         f_new.write(line)
                    if begin_config:
                         line_0 = line.replace('\t','').replace('\n','') #Removing junk
                         if len(line_0)>1:
                              if line_0[0] =='#': 'Continuing'; continue #Ignoring lines which are commented out
                         line_3 = line_0.replace("'",'').split(':')[0]
                         line_4 = line_0.strip().split(':')[0].replace("'",'')
                         if '{' in line_0:
                              key_tracker.append(line_4)
                              print(key_tracker)
                         if '}' in line_0:
                              assert line_0.replace(',','').strip()=='}'
                              key_tracker = key_tracker[:-1]
                              print(key_tracker)
                         try:
                              #If the update-dictionary contains these keys (and the corresponding value is a float), the 
                              #value needs to be updated
                              if getFromDict(new_config_dict_values,key_tracker+[line_3]) is None:
                                   f_new.write(str(line_0)+'\n')
                              elif isinstance(getFromDict(new_config_dict_values,key_tracker+[line_3]),float):
#                                   print('Writing',"'"+line_3+"':"+
#                                   str(self.getFromDict(self.new_config_dict_values,key_tracker+[line_3]))+",\n")
                                   f_new.write("'"+line_3+"':"+
                                   str(getFromDict(new_config_dict_values,key_tracker+[line_3]))+",\n")
                                   N_values_changed+=1
                              elif isinstance(getFromDict(new_config_dict_values,key_tracker+[line_3]),str):
                                   print('Writing',"'"+line_3+"':"+
                                   str(getFromDict(new_config_dict_values,key_tracker+[line_3]))+",\n")
                                   f_new.write("'"+line_3+"':"+
                                   str(getFromDict(new_config_dict_values,key_tracker+[line_3]))+",\n")
                                   N_values_changed+=1
                              else:
                                   print('Else',line_0,type(getFromDict(new_config_dict_values,key_tracker+[line_3])))
                         except Exception as ex:
                              print('EX',ex,line_0)
                              f_new.write(str(line_0)+'\n')
                              pass
                    if 'config_dict' in line: begin_config=True
          print(N_values_changed,N_new_values)
          assert N_values_changed==N_new_values #Assert that the same number of value have been updated as in the new_config_dict_values dictionary.  
     write_new_config()

In [4]:
import pandas as pd
LensPop_config_db = pd.read_csv('/mnt/zfsusers/hollowayp/sim-pipeline/notebooks/Lenspop_config_db.csv')
Simpipeline_config_db = pd.read_csv('/mnt/zfsusers/hollowayp/sim-pipeline/notebooks/simpipeline_config_db.csv')
LensPop_config_db.columns

Index(['Unnamed: 0', 'prop', 'mean', 'std', 'sigma_to_zero', 'trunc_bool'], dtype='object')

In [11]:
import numpy as np
generate_config_file_based_on_database('/mnt/zfsusers/hollowayp/paltas/paltas/Configs/Examples/config_LSST.py',
                                       '/mnt/zfsusers/hollowayp/paltas/paltas/Configs/Examples/config_Lenspop.py',
                                        list(LensPop_config_db['prop']),
                                        list(LensPop_config_db['mean']),
                                        list(LensPop_config_db['std']),
                                        list(LensPop_config_db['sigma_to_zero']),
                                        list(LensPop_config_db['trunc_bool']))

generate_config_file_based_on_database('/mnt/zfsusers/hollowayp/paltas/paltas/Configs/Examples/config_LSST.py',
                                       '/mnt/zfsusers/hollowayp/paltas/paltas/Configs/Examples/config_Simpipeline.py',
                                        list(Simpipeline_config_db['prop']),
                                        list(Simpipeline_config_db['mean']),
                                        list(Simpipeline_config_db['std']),
                                        list(Simpipeline_config_db['sigma_to_zero']),
                                        list(Simpipeline_config_db['trunc_bool']))


{'main_deflector': {'parameters': {'z_lens': 'truncnorm(-1.9316781735341293,np.inf,loc=0.4877719913959017,scale=0.2525120375013046).rvs', 'theta_E': 'truncnorm(-3.854034453681824,np.inf,loc=1.415142080833239,scale=0.3671845952184807).rvs', 'center_x': 0.0, 'center_y': 0.0, 'gamma': 2.0, 'e1': 0.0, 'e2': 0.0, 'gamma1': 0.0, 'gamma2': 0.0}}, 'lens_light': {'parameters': {'z_source': 'truncnorm(-1.9316781735341293,np.inf,loc=0.4877719913959017,scale=0.2525120375013046).rvs', 'mag_app': 'truncnorm(-10.341669207413242,np.inf,loc=18.29812747650855,scale=1.7693591923624732).rvs', 'output_ab_zeropoint': None, 'R_sersic': 'truncnorm(-1.8573312405609943,np.inf,loc=0.9325585871164952,scale=0.5020960002992368).rvs', 'n_sersic': 4.0, 'e1': 0.0, 'e2': 0.0, 'center_x': 0.0, 'center_y': 0.0}}, 'source': {'parameters': {'z_source': 'truncnorm(-2.463209501776144,np.inf,loc=2.130605796445149,scale=0.8649714102307722).rvs', 'mag_app': 'truncnorm(-23.112963222304234,np.inf,loc=25.59522698969772,scale=1.107